In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
mainURL = "https://www.programiz.com/c-programming"

page = requests.get(mainURL)
htmlContent = page.content
soup = BeautifulSoup(htmlContent, 'html.parser')

urls = []
pages = soup.find('div', {'class': 'right-bar'}).findAll(href=True)
for slug in pages[:-7]:
    urls.append('https://www.programiz.com' + slug.get('href'))

mainHeadings = []
mainContents = []
videoHeadings = []
videoURLs = []
subHeadingsH2 = []
subHeadingsH3 = []
imageURLs = []
facts = []
definitions = []
explanations = []
codes = []
snippets = []
outputs = []
tableData = []
listItems = []
data = {}


In [3]:
for url in urls:
    page = requests.get(url)
    htmlContent = page.content
    soup = BeautifulSoup(htmlContent, 'html.parser')

    mainHeading = soup.findAll('h1')  # Main Heading
    pageMainHeadings = []
    if mainHeading is not None:
        for head in mainHeading:
            pageMainHeadings.append(head.text)

    mainContent = soup.findAll('p', {"class": "editor-contents__short-description"})  # Main Content
    pageMainContents = []
    if mainContent is not None:
        for mainCon in mainContent:
            pageMainContents.append(mainCon.text)

    videoHeading = soup.findAll('div', {"class": "programiz-youtube-wrapper lite-youtube"})  # Video Heading
    pageVideoHeadings = []
    if videoHeading is not None:
        for vidHead in videoHeading:
            pageVideoHeadings.append(vidHead.find('h2').text)

    videoURL = soup.findAll('div', {"class": "programiz-youtube__container"})  # Video URL
    pageVideoURLs = []
    if videoURL is not None:
        for vid in videoURL:
            pageVideoURLs.append(vid.find('iframe').get('src'))

    subContent = soup.find('div', {"class": "content"})  # Sub Content
    if subContent is None:
        continue

    subHeadingH2 = subContent.findAll('h2')  # Sub Headings h2
    pageSubHeadingsH2 = []
    if subHeadingH2 is not None:
        for head in subHeadingH2:
            pageSubHeadingsH2.append(head.text)

    subHeadingH3 = subContent.findAll('h3')  # Sub Headings h3
    pageSubHeadingsH3 = []
    if subHeadingH3 is not None:
        for head in subHeadingH3:
            pageSubHeadingsH3.append(head.text)

    images = subContent.findAll('img')  # Images
    pageImageURLs = []
    if images is not None:
        for img in images:
            pageImageURLs.append(img.get('src'))

    paragraphs = subContent.findAll('p')  # Paragraphs
    pageFacts = []
    pageDefinitions = []
    pageExplanations = []
    if paragraphs is not None:
        for para in paragraphs:
            slug = para.text
            if 5 <= len(slug) <= 65:
                pageFacts.append(slug)
            elif 66 <= len(slug) <= 126:
                pageDefinitions.append(slug)
            elif 127 < len(slug):
                pageExplanations.append(slug)

    codeContent = subContent.findAll('pre')  # Codes, Snippets and Outputs
    pageCodes = []
    pageSnippets = []
    pageOutputs = []
    for slug in codeContent:
        if slug.code is not None:
            for cod in slug.code.text.split('\n'):
                if cod != '' and not cod.lstrip().rstrip().startswith('//'):
                    if 7 < len(cod) <= 30:
                        pageSnippets.append(cod)
                    elif 31 < len(cod):
                        pageCodes.append(cod)

        else:
            for out in slug.text.split('\n'):
                if out != '' and not out.lstrip().rstrip().startswith('//'):
                    pageOutputs.append(out)
                else:
                    continue

    tables = subContent.findAll('table')  # Tables
    pageTableData = []
    if tables is not None:
        for tb in tables:
            table = ""
            rowData = []
            tableRows = tb.findAll('tr')
            for row in tableRows:
                rowData.append(row.text[1:-1].replace('\n', '|||'))
            for row in rowData:
                table = table + str(row) + '&&&'
            pageTableData.append(table[:-3])

    allListUL = subContent.findAll('ul')  # List Items
    allListOL = subContent.findAll('ol')
    listUL = ""
    listOL = ""
    listItem = ""
    pageListItems = []
    if len(allListUL) != 0 or len(allListOL) != 0:
        if len(allListUL) != 0:
            for lis in allListUL:
                listUL = listUL + str(lis.text).replace('\n', '|||').replace('\t\t', '')
        if len(allListOL) != 0:
            for lis in allListOL:
                listOL = listOL + str(lis.text).replace('\n', '|||').replace('\t\t', '')

        if len(allListUL) != 0 and len(allListOL) == 0:
            listItem = listItem + listUL
        elif len(allListUL) == 0 and len(allListOL) != 0:
            listItem = listItem + listOL
        else:
            listItem = listItem + listUL + '&&&' + listOL
        pageListItems.append(listItem)

    mainHeadings.append(pageMainHeadings)
    mainContents.append(pageMainContents)
    videoHeadings.append(pageVideoHeadings)
    videoURLs.append(pageVideoURLs)
    subHeadingsH2.append(pageSubHeadingsH2)
    subHeadingsH3.append(pageSubHeadingsH3)
    imageURLs.append(pageImageURLs)
    facts.append(pageFacts)
    definitions.append(pageDefinitions)
    explanations.append(pageExplanations)
    snippets.append(pageSnippets)
    codes.append(pageCodes)
    outputs.append(pageOutputs)
    tableData.append(pageTableData)
    listItems.append(pageListItems)


In [4]:
data = {'main_headings': mainHeadings, 'main_contents': mainContents, 'video_headings': videoHeadings,
        'video_urls': videoURLs, 'sub_headings_h2': subHeadingsH2, 'sub_headings_h3': subHeadingsH3,
        'image_urls': imageURLs, 'facts': facts, 'definitions': definitions, 'explanations': explanations,
        'snippets': snippets, 'codes': codes, 'outputs': outputs, 'table_data': tableData, 'list_items': listItems}

df = pd.DataFrame(data)
df.to_csv('Programiz_C.csv')

In [5]:
a=pd.read_csv("Programiz_C.csv")
a

,Unnamed: 0,main_headings,main_contents,video_headings,video_urls,sub_headings_h2,sub_headings_h3,image_urls,facts,definitions,explanations,snippets,codes,outputs,table_data,list_items
0,0,['C Keywords and Identifiers'],"['In this tutorial, you will learn about keywo...",[],[],"['Character set', 'C Keywords', 'C Identifiers']","['Alphabets', 'Digits', 'Special Characters', ...",[],"['White space Characters', 'Here, money and ac...","['A character set is a set of alphabets, lette...","['Keywords are predefined, reserved words used...","['int money;', 'int money;', 'double accountBa...",[],['Uppercase: A B C ..............................,['|||<|||>|||.|||_&&&|||)|||;|||$|||:&&&|||[||...,['A valid identifier can have letters (both up...
1,1,"['C Variables, Constants and Literals']","['In this tutorial, you will learn about varia...",['Video: Variables in C Programming'],['https://www.youtube.com/embed/h4VBpylsjJc?li...,"['Variables', 'Literals', 'Constants']","['Rules for naming a variable', '1. Integers',...",[],"['The value of a variable can be changed, henc...","['In programming, a variable is a container (s...","['To indicate the storage area, each variable ...","['int playerScore = 95;', ""char ch = 'a';"", ""c...",['int number = 5; // integer variable'],"['Decimal: 0, -9, 22 etc', 'Octal: 021, 077, 0...",['scape Sequences|||Character&&&b|||Backspace&...,['decimal\xa0(base 10)|||octal (base 8)|||hexa...
2,2,['C Data Types'],"['In this tutorial, you will learn about basic...",['Video: Data Types in C Programming'],['https://www.youtube.com/embed/sARaqR0hRI4?li...,"['Basic types', 'Derived Data Types']","['int', 'float and double', 'char', 'void', 's...",[],['We can use int for declaring an integer vari...,"['Here, myVar is a variable of int (integer) t...","['In C programming, data types are declaration...","['int myVar;', 'int id, age;', 'float salary;'...","['float normalizationFactor = 22.442e2;', ' p...",['short d;'],['ype|||Size (bytes)|||Format Specifier&&&nt||...,['signed - allows for storage of both positive...
3,3,['C Input Output (I/O)'],"['In this tutorial, you will learn to use scan...",['Video: Get User Input in C Programming'],['https://www.youtube.com/embed/17gp5DJEyiw?li...,"['C Output', 'C Input', 'I/O Multiple Values',...","['Example 1: C Output', 'Example 2: Integer Ou...",[],"['Output', 'How does this program work?', 'Out...","['In C programming, printf() is one of the mai...","['In C programming, scanf() is one of the comm...","['#include <stdio.h> ', 'int main()', ' ...","[' printf(""Number = %d"", testInteger);', ' ...","['C Programming', 'Number = 5', 'number1 = 13....",['ata Type|||Format Specifier&&&nt|||%d&&&har|...,['All valid C programs must contain the main()...
4,4,['C Programming Operators'],"['In this tutorial, you will learn about diffe...",['Video: Arithmetic Operators in C'],['https://www.youtube.com/embed/_57FcSBtJNU?li...,"['C Arithmetic Operators', 'C Increment and De...","['Example 1: Arithmetic Operators', 'Example 2...",[],['C has a wide range of operators to perform v...,['An operator is a symbol that operates on a v...,['An arithmetic operator performs mathematical...,"['#include <stdio.h>', 'int main()', ' int ...","[' printf(""Remainder when a divided by b = ...","['a+b = 13', 'a-b = 5', 'a*b = 36', 'a/b = 2',...",['perator|||Meaning of Operator&&&|||addition ...,['(a == b) && (c > 5) evaluates to 1 because b...
5,5,['C Introduction Examples'],"['In this article, you will find a list of sim...",[],[],['Examples'],[],[],['We have learned about the following topics s...,"['Before you go through these examples, we sug...",['We understand that programming can by start ...,[],[],[],[' program to print a sentence&&& program to p...,['Variables and Constants|||Data Types|||Input...
6,6,['C if...else Statement'],"['In this tutorial, you will learn about the i...",['Video: C if else Statement'],['https://www.youtube.com/embed/K8mntKyBJGc?li...,"['C if Statement', 'C if...else Statement', 